# Import

In [12]:
import os
os.add_dll_directory(r"C:\msys64\mingw64\bin")  # phải đặt TRƯỚC khi import cairosvg/cairocffi


<AddedDllDirectory('C:\\msys64\\mingw64\\bin')>

In [13]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, Any, Tuple, Optional, List
from pathlib import Path

from moviepy import ImageClip, ColorClip, CompositeVideoClip, VideoClip
import io, numpy as np
from pathlib import Path
from PIL import Image


# Data Class

In [2]:
Anchor = str  # "top_left" | "top_right" | "bottom_left" | "bottom_right" | "center"
Size = Tuple[int, int]

@dataclass
class Chip:
    enabled: bool = False
    opacity: float = 0.28
    radius: int = 14         # bo góc chip (dùng mask đơn giản)
    pad_px: int = 8          # nới chip lớn hơn logo (mỗi cạnh)

@dataclass
class LogoOpts:
    anchor: Anchor = "top_right"
    inset: int = 48
    width_pct: float = 0.07
    max_height_pct: float = 0.08
    opacity: float = 0.8
    chip: Chip = field(default_factory=Chip)

# Ultils

In [16]:
# thêm vào đầu file
def _imageclip_from_asset(path: str, target_w: int | None = None, target_h: int | None = None):
    """Trả về ImageClip từ PNG/JPG hoặc SVG (rasterize bằng cairosvg)."""
    if path.lower().endswith(".svg"):
        import cairosvg
        svg_bytes = Path(path).read_bytes()
        if target_w:
            png_bytes = cairosvg.svg2png(bytestring=svg_bytes, output_width=int(target_w))
        elif target_h:
            png_bytes = cairosvg.svg2png(bytestring=svg_bytes, output_height=int(target_h))
        else:
            png_bytes = cairosvg.svg2png(bytestring=svg_bytes, output_width=1024)
        img = Image.open(io.BytesIO(png_bytes)).convert("RGBA")
        return ImageClip(np.array(img))
    else:
        return ImageClip(path)


# Geometry

In [ ]:
def _anchor_xy(anchor: Anchor, logo_wh: Size, canvas_wh: Size, inset: int) -> Tuple[int, int]:
    lw, lh = logo_wh; W, H = canvas_wh
    if anchor == "top_left":     return inset, inset
    if anchor == "top_right":    return W - inset - lw, inset
    if anchor == "bottom_left":  return inset, H - inset - lh
    if anchor == "bottom_right": return W - inset - lw, H - inset - lh
    return (W - lw)//2, (H - lh)//2  # center

def _scale_logo(clip: ImageClip, canvas_wh: Size, width_pct: float, max_height_pct: float) -> ImageClip:
    W, H = canvas_wh
    # scale theo width trước
    target_w = max(1, int(W * width_pct))
    scaled = clip.resized(new_size=(target_w, int(clip.h * (target_w / clip.w))))
    # giới hạn theo max_height
    max_h = int(H * max_height_pct)
    if scaled.h > max_h:
        scaled = scaled.resized(new_size=(int(scaled.w * (max_h / scaled.h)), max_h))
    return scaled

def _intersect(a: Tuple[int,int,int,int], b: Tuple[int,int,int,int]) -> bool:
    ax, ay, aw, ah = a; bx, by, bw, bh = b
    return not (bx > ax+aw or bx+bw < ax or by > ay+ah or by+bh < ay)

def _resolve_avoid_rects(canvas_wh: Size, avoid_specs: List[Dict[str, Any]]) -> List[Tuple[int,int,int,int]]:
    W, H = canvas_wh
    rects = []
    for it in avoid_specs or []:
        if "rect_pct" in it and it["rect_pct"] and len(it["rect_pct"]) == 4:
            x, y, w, h = it["rect_pct"]
            rects.append((int(x*W), int(y*H), int(w*W), int(h*H)))
        elif "rect_px" in it and it["rect_px"] and len(it["rect_px"]) == 4:
            rects.append(tuple(it["rect_px"]))
    return rects

def _auto_pick_anchor(initial: Anchor, order: List[Anchor], logo_wh: Size, canvas_wh: Size,
                      inset: int, avoid_rects: List[Tuple[int,int,int,int]]) -> Anchor:
    # thử theo thứ tự ưu tiên, chọn anchor không va chạm
    for anchor in ([initial] + [a for a in order if a != initial]):
        x, y = _anchor_xy(anchor, logo_wh, canvas_wh, inset)
        rect = (x, y, logo_wh[0], logo_wh[1])
        if not any(_intersect(rect, ar) for ar in avoid_rects):
            return anchor
    return initial


In [5]:
# ---------- chip (nền mờ sau logo) ----------
def _chip_clip(size: Size, chip: Chip) -> Optional[VideoClip]:
    if not chip.enabled: return None
    w, h = size
    # ColorClip không có radius; ta đơn giản dùng clip hình chữ nhật mờ
    # (Nếu cần bo góc thật: tạo mask RGBA bằng PIL rồi ImageClip)
    bg = ColorClip(size=(w + chip.pad_px*2, h + chip.pad_px*2), color=(0,0,0)).with_opacity(chip.opacity)
    return bg


In [21]:

# ---------- build logo ----------
def build_logo_clip(
    spec: Dict[str, Any],
    variant: str,                 # "primary_lockup" | "alternate_horizontal" | "alternate_stacked" | "mark_only" | "wordmark_only"
    mode: str = "watermark",      # "watermark" | "hero"
    canvas_size: Size = (1920, 1080),
    duration: Optional[float] = None,
    overrides: Optional[Dict[str, Any]] = None,
    extra_avoid_rects_px: Optional[List[Tuple[int,int,int,int]]] = None
) -> VideoClip:
    """
    Trả về ImageClip đã scale, đặt anchor & inset, áp opacity, chip nền, auto-avoid (nếu có).
    Clip trả về đã .with_position((x,y)), không có background (để ghép CompositeVideoClip).
    """
    overrides = overrides or {}
    cfg = spec["logo"]

    # 1) Lấy đường dẫn asset
    assets = cfg["assets"]
    if variant not in assets:
        raise ValueError(f"Variant '{variant}' không tồn tại trong spec.assets")
    src = assets[variant]
    if not Path(src).exists():
        raise FileNotFoundError(f"Không tìm thấy file logo: {src}")

    # 2) Build preset (global preset + per-variant override + overrides)
    preset = dict(cfg["presets"][mode])  # shallow copy
    per = cfg.get("per_variant_overrides", {}).get(variant, {})
    if mode in per:
        # merge override cho preset cùng mode
        def _deepmerge(a, b):
            out = dict(a)
            for k, v in b.items():
                if isinstance(v, dict) and isinstance(out.get(k), dict):
                    out[k] = _deepmerge(out[k], v)
                else:
                    out[k] = v
            return out
        preset = _deepmerge(preset, per[mode])

    # apply overrides cuối cùng (ưu tiên cao nhất)
    for k, v in overrides.items():
        if k == "chip" and isinstance(v, dict):
            preset["chip"] = {**preset.get("chip", {}), **v}
        else:
            preset[k] = v

    # 3) Parse options
    chip_cfg = preset.get("chip", {}) or {}
    chip = Chip(
        enabled=bool(chip_cfg.get("enabled", False)),
        opacity=float(chip_cfg.get("opacity", 0.28)),
        radius=int(chip_cfg.get("radius", 14)),
        pad_px=int(chip_cfg.get("pad_px", 8)),
    )
    opts = LogoOpts(
        anchor=preset.get("anchor", "top_right"),
        inset=int(preset.get("inset", 48)),
        width_pct=float(preset.get("width_pct", 0.07)),
        max_height_pct=float(preset.get("max_height_pct", 0.08)),
        opacity=float(preset.get("opacity", 0.8)),
        chip=chip
    )

    # 4) Load & scale (ưu tiên rasterize đúng kích thước mục tiêu để nét)
    W, H = canvas_size
    target_w = max(1, int(W * opts.width_pct))
    max_h = int(H * opts.max_height_pct)

    logo = _imageclip_from_asset(src, target_w=target_w)  # PNG/JPG đọc thẳng, SVG → PNG in-memory
    if duration is not None:
        logo = logo.with_duration(duration)
    # Nếu cao quá ngưỡng, co theo chiều cao
    if logo.h > max_h:
        logo = logo.resized(new_size=(int(logo.w * (max_h / logo.h)), max_h))


    # 5) Optional chip
    local_layers: List[VideoClip] = []
    chip_clip = _chip_clip((logo.w, logo.h), opts.chip)
    if chip_clip is not None:
        local = CompositeVideoClip([chip_clip, logo.with_position((opts.chip.pad_px, opts.chip.pad_px))],
                                   size=(chip_clip.w, chip_clip.h))
        logo = local  # treat as a single clip of (logo + chip)

    # 6) Auto-avoid collisions
    avoid_rects = _resolve_avoid_rects(canvas_size, cfg.get("collision_rules", {}).get("avoid", []))
    if extra_avoid_rects_px:
        avoid_rects.extend(extra_avoid_rects_px)
    anchor = opts.anchor
    if cfg.get("collision_rules", {}).get("auto_flip", True):
        order = cfg.get("collision_rules", {}).get("priority_order",
                    ["top_right","top_left","bottom_right","bottom_left","center"])
        anchor = _auto_pick_anchor(anchor, order, (logo.w, logo.h), canvas_size, opts.inset, avoid_rects)

    # 7) Position theo anchor & inset
    x, y = _anchor_xy(anchor, (logo.w, logo.h), canvas_size, opts.inset)
    return logo.with_position((x, y))


# Test

In [ ]:
import requests
# open('logo.png','wb').write(requests.get('https://w7.pngwing.com/pngs/857/23/png-transparent-logo.png').content)

4546

In [26]:
heroes

In [27]:
import json
from moviepy import ColorClip, CompositeVideoClip, concatenate_videoclips

# 1) Load spec
with open("D:/ThienPV/code/demo/assets/logo_spec_v1.json", "r", encoding="utf-8") as f:
    SPEC = json.load(f)

W,H = 1920,1080
bg = ColorClip((W,H), color=(15,18,24)).with_duration(5)

# 3) Hero lockup (center) 2s đầu (chèn vào intro, tuỳ chọn)
HERO_SPEC = SPEC
heroes = []
types = ["primary_lockup", "alternate_horizontal", "alternate_stacked", "mark_only", "wordmark_only"]
for type in types:
    for mode in ["hero", "watermark"]:
        HERO_SPEC["logo"]["assets"][type]= "D:/ThienPV/code/demo/data/logo.png"
        hero = build_logo_clip(
            HERO_SPEC,
            variant=type,
            mode=mode,
            canvas_size=(W,H),
            duration=1
        ).with_start(0.0)
        final = CompositeVideoClip([bg,hero]) 
        heroes.append(final)
for hero in heroes:
    hero.preview()

OSError: [Errno 22] Invalid argument

MoviePy error: FFPLAY encountered the following error while previewing clip :

 None

In [23]:
final.preview()

OSError: [Errno 22] Invalid argument

MoviePy error: FFPLAY encountered the following error while previewing clip :

 None